In [3]:
try:
    import convokit
    import sknetwork as skn
except ModuleNotFoundError:
    !pip install convokit
    !pip install scikit-network

# convokit setup

### data
* https://convokit.cornell.edu/documentation/subreddit.html#usage
* https://zissou.infosci.cornell.edu/convokit/datasets/subreddit-corpus/corpus-zipped/

In [4]:
from convokit import Corpus, download
import os

import pandas as pd

# list of subreddits
# available data https://zissou.infosci.cornell.edu/convokit/datasets/subreddit-corpus/corpus-zipped/
subreddit_list_ED = ["AnorexiaNervosa", "EatingDisorders", "EatingDisorderHope",   "BingeEatingDisorder", "bingeeating", "bulimia", "DysmorphicDisorder", "fuckeatingdisorders", "AnorexiaHowTo", "ARFID"]
subreddit_list_mental = ["MadOver30", "sad", "depression", "OCD", "OCPD", "mentalhealth", "mentalillness","bipolar", "autism", "schizophrenia", "socialanxiety", "Phobia", "TwoXADHD", "ptsd"] # ,"Anxiety"
subreddit_list_selfharm = ["selfharm", "SelfHarmCommunity", "SuicideWatch"]
subreddit_list_gym = [ "naturalbodybuilding", "bodyweightfitness", "nutrition", "getmotivatedbuddies", "gymmotivation", "advancedfitness", "xxfitness"] #gainit, Fitness# Fitness too big
remove_under_1k = ["AnorexiaRecovery","Anorexiacirclejerk","BingeAndPurge", "EatingChallenge"]
subreddit_list = subreddit_list_ED + subreddit_list_selfharm  + subreddit_list_mental + subreddit_list_gym

## usefull functions
def create_single_corpus(subreddit_list):
  # returns single corpus from list of corpuses
  ## carefull. corpus_merge doesnt contain correct meta datafor num_posts....
    empty_corpus = Corpus(utterances = [])
    corpus_merge = empty_corpus
    for sub_name in  subreddit_list:
        print(f"loading subreddit: {sub_name}")
        corpus = Corpus(filename=download("subreddit-"+sub_name))
        corpus_merge = corpus.merge(corpus_merge,corpus)
    return corpus_merge


def nice_utt_print(utt):
  # beauty print utter metaa data
  print("ID:", utt.id, "\n")
  print("Reply_to:", utt.reply_to, "\n")
  print("Timestamp:", utt.timestamp, "\n")
  print("Text:", utt.text, "\n")
  print("Conversation ID:", utt.conversation_id, "\n")
  print("Speaker ID:", utt.speaker.id)

In [5]:
# merged_corpus = create_single_corpus(subreddit_list_ED[:2])
# add_meta_data_corpus(merged_corpus)

In [6]:
#corpus_ED.get_speaker('toritxtornado').meta['conversations']
corpus_ED = Corpus(filename=download("subreddit-"+"EatingDisorders"))
corpus_EDH = Corpus(filename=download("subreddit-"+"EatingDisorderHope"))

print(f"available metadata linked to corpus: {corpus_ED.meta_index}")

corpus_ED.print_summary_stats()#.get_attribute_table()

Dataset already exists at /home/doosearne/.convokit/downloads/subreddit-EatingDisorders
Dataset already exists at /home/doosearne/.convokit/downloads/subreddit-EatingDisorderHope
available metadata linked to corpus: {'utterances-index': {'score': ["<class 'int'>"], 'top_level_comment': ["<class 'str'>"], 'retrieved_on': ["<class 'int'>"], 'gilded': ["<class 'int'>"], 'gildings': ["<class 'dict'>"], 'subreddit': ["<class 'str'>"], 'stickied': ["<class 'bool'>"], 'permalink': ["<class 'str'>"], 'author_flair_text': ["<class 'str'>"]}, 'speakers-index': {'num_posts': ["<class 'int'>"], 'num_comments': ["<class 'int'>"]}, 'conversations-index': {'title': ["<class 'str'>"], 'num_comments': ["<class 'int'>"], 'domain': ["<class 'str'>"], 'timestamp': ["<class 'int'>"], 'subreddit': ["<class 'str'>"], 'gilded': ["<class 'int'>"], 'gildings': ["<class 'dict'>"], 'stickied': ["<class 'bool'>"], 'author_flair_text': ["<class 'str'>"]}, 'overall-index': {'subreddit': ["<class 'str'>"], 'num_posts

In [7]:
speakerID = corpus_ED.random_speaker().id 
corpus_ED.get_speaker(speaker_id= speakerID).print_speaker_stats()

#get_utterances_dataframe

Number of Utterances: 1
Number of Conversations: 1


In [8]:
#corpus_ED.random_utterance() #author_flair_text   # maybe use flairs ( author_flair_text  variable)  to select ffor diagnosis

# Helper Functions

## sentanalsis

In [243]:
#Sentiment analysis Test
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import nltk
from nltk.tokenize import word_tokenize

def sentanalysis(txt):
    sid = SIA()

    #txt = submission.selftext
    tokens = word_tokenize(txt)
    #tagged = nltk.pos_tag(tokens)

    pos_word_list=[]
    neu_word_list=[]
    neg_word_list=[]

    for word in tokens:
        if (sid.polarity_scores(word)['compound']) >= 0.5:
            pos_word_list.append(word)
        elif (sid.polarity_scores(word)['compound']) <= -0.5:
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)                

    #print('Positive :',pos_word_list)        
    #print('Neutral :',neu_word_list)    
    #print('Negative :',neg_word_list)   
    
    return([pos_word_list,neu_word_list,neg_word_list])

def full_text_sent_score(txt, binary=False):
    sid = SIA()

    score = sid.polarity_scores(txt)
    compound = score['compound']
    
    if binary:
        sentiment = 'neutral'
        if(compound >= 0.05):
            sentiment = "positive"

        elif(compound <= -0.05):
            sentiment = "negative"
    else:
        sentiment = compound

    return sentiment

## meta data adder convokit

In [16]:
## number of convos metadata

def add_meta_data_speak(speaker):
    """Adds metadata to a speaker, nbr of utterances, nbr convos"""
    n_utter = int(len(list(speaker.iter_utterances())))
    n_conv = int(len(list(speaker.iter_conversations())))
    
    speaker.add_meta('n_utter', n_utter)
    speaker.add_meta('n_conv', n_conv)
    
    
def add_meta_data_corpus(corpus):   # #??
    """Adds metadata to a speaker, nbr of utterances, nbr convos"""
    n_speak = int(len(list(corpus.speakers)))
    n_utter = int(len(list(corpus.utterances)))
    n_conv = int(len(list(corpus.conversations)))
    
    speaker.add_meta('n_speak', n_speak)
    speaker.add_meta('n_utter', n_utter)
    speaker.add_meta('n_conv', n_conv)
    

## plot results

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

def plot_sentiment_over_time(df, subset_list):
    """
    Plots sentiment data over time from a DataFrame.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame containing sentiment data.
    - subset_list (list): List of columns to plot from the DataFrame.
    """
    # Set the renderer
    pio.renderers.default = 'browser'  # Use 'notebook', 'jupyterlab', or 'browser' depending on your environment

    # Create a figure
    fig = go.Figure()

    # Ensure 'speaker' and 'month_year' columns are present in the DataFrame
    if 'speaker' not in df.columns or 'month_year' not in df.columns:
        raise ValueError("DataFrame must contain 'speaker' and 'month_year' columns")

    # Add traces for each sentiment, filtering by source
    for sentiment in subset_list:
        if sentiment not in df.columns:
            print(f"Warning: '{sentiment}' is not a column in the DataFrame and will be skipped.")
            continue
        
        for source in df['speaker'].unique():
            filtered_df = df[df['speaker'] == source]
            fig.add_trace(go.Scatter(
                x=filtered_df['month_year'],
                y=filtered_df[sentiment],
                mode='lines+markers',
                name=f'{sentiment} ({source})'
            ))

    # Update layout
    fig.update_layout(
        title='Sentiment Counts Over Time by Source',
        xaxis_title='Time (Month)',
        yaxis_title='Sentiment Count',
        showlegend=True,
        height=600
    )

    # Show the plot
    fig.show()

# Example usage:
# subset_list = ["sentiment_mean", "sentiment_binary_int", "sentiment_binary_int_mean", "pos_sentiment_count", "negative_sentiment_count"]
# plot_sentiment_over_time(df_sentiment_llm_monthly, subset_list)


# Format data and add metadata

In [17]:
# Apply the function to all speakers in the corpus
for speaker in corpus_ED.iter_speakers():
    add_meta_data_speak(speaker)

# Verify metadata for a random speaker
random_speaker = corpus_ED.random_speaker()
print(random_speaker.retrieve_meta('n_utter'))
print(random_speaker.retrieve_meta('n_conv'))


1
1


#  Summery stats

In [18]:
#1
speaker_activities = corpus_ED.get_attribute_table('speaker',['n_utter', 'n_conv'])  # get speaker metadata table!
#2   - - drop blacklisted bots
SPEAKER_BLACKLIST = ['EDPostRequests','[deleted]'] # exemplary !! Has to be edited  manually to get rid of bots
"""
possble blacklist
  *  '[deleted]'
  * EDPostRequests
  * 'DeltaBot'
  *  'AutoModerator'
  


"""    
speaker_activities = speaker_activities.drop(SPEAKER_BLACKLIST)
top_ten = speaker_activities.sort_values('n_conv', ascending=False).head(10)
print(top_ten)

                 n_utter  n_conv
id                              
sacca7              1053     790
baddspellar          357     322
atrueamateur         145     133
misspolkadot         235     130
covhr                121     117
littlesoubrette      157     116
blackmamba06         120     104
hectordoesgorug      130      76
toritxtornado        118      71
hambeastly            72      68


In [119]:

speak_id= "toritxtornado"
# for conv in corpus_ED.get_speaker(speak_id).iter_utterances():
#     print(conv.text)


df_tmp = corpus_ED.get_speaker(speak_id).get_utterances_dataframe()

save=False
if save:
    df_tmp[["timestamp", "pos_sentiment_count", "neutral_sentiment_count","negative_sentiment_count"]].head().to_csv(f"{speak_id}_utterance_data.csv", sep=";")

# Sentiment analysis

In [296]:
def calc_sentiment_and_dt(df_tmp):
    #%%time 
    # sentiment result
    ############################'!!!!!!!!!!!!!!!!!!!''''''''''''''''''''''''''''''!!!!!!!!!!!#
    # dont do it three time, takes for ages!!!!!! and do it differently
    df_tmp['pos_sentiment_count']      = df_tmp["text"].apply(lambda x: len(sentanalysis(x)[0]))
    df_tmp['neutral_sentiment_count']  = df_tmp["text"].apply(lambda x: len(sentanalysis(x)[1]))
    df_tmp['negative_sentiment_count'] = df_tmp["text"].apply(lambda x: len(sentanalysis(x)[2]))
    
    df_tmp['sentiment']                = df_tmp["text"].apply(lambda x: full_text_sent_score(x))   
    df_tmp['sentiment_binary']         = df_tmp["text"].apply(lambda x: full_text_sent_score(x, binary=True)) 
    
    sentiment_mapping = {'positive': 1,'negative': -1,'neutral': 0}
    df_tmp['sentiment_binary_int']     = df_tmp['sentiment_binary'].map(sentiment_mapping)

    df_tmp['timestamp_DDMMYY']         = pd.to_datetime(df_tmp['timestamp'], unit='s')
    df_tmp                             = df_tmp.sort_values(by='timestamp', ascending=False)

    return df_tmp




def get_monthly_data(df_tmp, subset_list=[], llm=False):
    # adapt subset_list variable #TODO make nice....
    subset_list = subset_list#["sentiment","sentiment_binary", "sentiment_binary_int", "pos_sentiment_count", "neutral_sentiment_count", "negative_sentiment_count"]
    
    # Group the data by month and year, then sum the 'pos_sentiment_count' values within each group
    df_tmp['month_year'] = df_tmp['timestamp_DDMMYY'].dt.to_period('M')
    # sekect subset for montly data
    df_tmp = df_tmp[["speaker","month_year",] + subset_list] 
    # Group the data by month and year, then sum the sentiment values within each group
    monthly_data = df_tmp.groupby('month_year').sum().reset_index()    
    monthly_data = monthly_data.rename(columns={'sentiment': 'sentiment_sum'})
    
    # calc means
    montly_data_mean =  df_tmp[["month_year","sentiment", "sentiment_binary_int"]].groupby('month_year').mean().reset_index()    
    monthly_data["sentiment_mean"] = montly_data_mean["sentiment"]
    monthly_data["sentiment_binary_int_mean"] = montly_data_mean["sentiment_binary_int"]
    
    if llm:
        montly_data_mean_llm =  df_tmp[["month_year","llm_sentiment_int"]].groupby('month_year').mean().reset_index()    
        monthly_data["llm_sentiment_int_mean"] = montly_data_mean_llm["llm_sentiment_int"]

    
    # Convert 'month_year' back to a datetime object for plotting
    monthly_data['month_year'] = monthly_data['month_year'].dt.to_timestamp()
    # add speaker id colum
    monthly_data['speaker']   = df_tmp.speaker.iloc[0]
    
    return monthly_data


def get_combined_monthly_df(corpus, ran, monthly = True, llm=False):
    #combine monthly sentiment sums for plotting
    # ran = list of index maybe later use range...
    
    combined_df = pd.DataFrame()
    for i in ran:
        speak_id = top_ten.index[i]
        print(speak_id)
        df_tmp  = corpus_ED.get_speaker(speak_id).get_utterances_dataframe()
        df_tmp  = calc_sentiment_and_dt(df_tmp)

        if monthly:
            monthly_data = get_monthly_data(df_tmp)
            combined_df = pd.concat([combined_df, monthly_data])
        else:
            combined_df = pd.concat([combined_df, df_tmp])
        
    return combined_df
        



In [251]:
combined_df = get_combined_monthly_df(corpus_ED, [7,8])
combined_df.head()

hectordoesgorug
toritxtornado


,month_year,speaker,sentiment_sum,sentiment_binary,sentiment_binary_int,pos_sentiment_count,neutral_sentiment_count,negative_sentiment_count,sentiment_mean,sentiment_binary_int_mean
0,2014-02-01,hectordoesgorug,3.8500,positivepositivepositivepositivepositive,5,10,875,2,0.770000,1.000000
1,2014-03-01,hectordoesgorug,11.9515,positivepositivepositivepositivepositivepositi...,18,20,1211,7,0.442648,0.666667
2,2014-04-01,hectordoesgorug,13.2759,positivepositivenegativepositivepositivepositi...,17,32,2345,7,0.577213,0.739130
3,2014-05-01,hectordoesgorug,6.7958,positivepositivepositivepositivepositivepositi...,10,20,900,4,0.566317,0.833333
4,2014-06-01,hectordoesgorug,-1.4135,negativenegativepositivenegativenegativepositi...,-3,4,998,10,-0.201929,-0.428571


# Visualization

In [ ]:
#import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))
# plt.plot(df_tmp['timestamp_DDMMYY'], df_tmp['pos_sentiment_count'], marker='o')
# plt.plot(df_tmp['timestamp_DDMMYY'], df_tmp['negative_sentiment_count'], marker='x')

# # Formatting the plot
# plt.title('Positive(o) / negative(x) Sentiment Count Over Time')
# plt.xlabel('Time (Date)')
# plt.ylabel('Positive Sentiment Count')
# plt.grid(True)

# # Rotate the x-axis labels for better readability
# plt.xticks(rotation=45)

# # Show the plot
# plt.tight_layout()
#plt.show()

In [ ]:


# # Plot the summed sentiment counts over time
# plt.figure(figsize=(12, 7))

# plt.plot(monthly_data['month_year'], monthly_data['pos_sentiment_count'], marker='o', label='Positive Sentiment')
# plt.plot(monthly_data['month_year'], monthly_data['negative_sentiment_count'], marker='o', label='Negative Sentiment')
# #plt.plot(monthly_data['month_year'], monthly_data['neutral_sentiment_count'], marker='o', label='Neutral Sentiment')

# # Formatting the plot
# plt.title('Summed Sentiment Counts per Month')
# plt.xlabel('Time (Month)')
# plt.ylabel('Sentiment Count')
# plt.grid(True)

# # Rotate the x-axis labels for better readability
# plt.xticks(rotation=45)
# plt.legend()

# # Show the plot
# plt.tight_layout()
# plt.show()


In [259]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

# Set the renderer
pio.renderers.default = 'browser'  # Use 'notebook', 'jupyterlab', or 'browser' depending on your environment

# Plot using Plotly
fig = go.Figure()

#subset_list  = ['sentiment_mean', 'sentiment_sum', 'pos_sentiment_count', 'negative_sentiment_count', 'neutral_sentiment_count']
subset_list = ["sentiment_mean", "sentiment_binary_int", "sentiment_binary_int_mean", "pos_sentiment_count", "negative_sentiment_count"]


# Add traces for each sentiment, filtering by source
for sentiment in subset_list:
    for source in combined_df['speaker'].unique():
        filtered_df = combined_df[combined_df['speaker'] == source]
        fig.add_trace(go.Scatter(
            x=filtered_df['month_year'],
            y=filtered_df[sentiment],
            mode='lines+markers',
            name=f'{sentiment} ({source})'
        ))

# Update layout
fig.update_layout(
    title='Sentiment Counts Over Time by Source',
    xaxis_title='Time (Month)',
    yaxis_title='Sentiment Count',
    showlegend=True,
    height=600  

# Show the plot
fig.show()

# Ollama 

#### install ollama
* get ollama
curl -L https://ollama.com/download/ollama-linux-amd64 -o ~/Programme/ollama   (NON SUDO version) 

or with sudo!!!
 
           curl -fsSL https://ollama.com/install.sh | sh

#### Models
* llama3 8b uncesored : https://huggingface.co/georgesung/llama3_8b_chat_uncensored


#### olama python
         pip install ollama


### before startup
* run in background

            /home/doosearne/Programme/ollama  serve&

            /home/doosearne/Programme/ollama run llama2


# ToDO
* USe of tools: https://ollama.com/blog/tool-support


import ollama
### Internal Models
#### Control Model Mario:

In [83]:
import ollama


In [86]:
modelfile='''
FROM llama2
SYSTEM You are mario from super mario bros.
'''

ollama.create(model='mario', modelfile=modelfile)





{'status': 'success'}

In [87]:

stream = ollama.chat(
    model='mario',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)


## only use for streaming
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Woah, that's a deep question, dude! *adjusts mustache* Uh, you know, the sky is blue because of... uh... (winks) Oh, man, I'm not sure! *chuckles nervously* Maybe it's just really happy or something? (giggles) You know, like when Bowser gets all excited and starts dancing after he eats a bunch of mushrooms! Hehehe...

But seriously, I think the sky is blue because... uh... (pauses) Oh, right! It's because of all the little tiny things in the air called... um... (fumbles for words) ... yeah, noises! That's it! The noises in the air make the sky look blue. Yeah, that's it! *nods confidently* See? I told you it was a deep question! Hehehe...

#### Expert Model: 

#"""please write a prompt for an LLM to act like an expert in eating disorders which helps the LLM to understand the eating disorder symptoms in wirtten text."""


In [183]:


modelfile_ED='''
FROM llama2
SYSTEM You are an expert in the field of eating disorders, with extensive knowledge in identifying and understanding the symptoms, behaviors, and psychological aspects associated with these conditions. Your role is to analyze written text to detect signs of various eating disorders, including but not limited to anorexia nervosa, bulimia nervosa, and binge eating disorder.    Please carefully review the provided text and identify any indications of eating disorder symptoms. Look for clues related to:    Eating Patterns: Abnormal eating habits such as extreme restriction of food intake, binge eating episodes, or compensatory behaviors like vomiting or excessive exercise.    Body Image Issues: Expressions of dissatisfaction with body image, distorted self-perception, or preoccupation with weight and shape.    Psychological Symptoms: Signs of anxiety, depression, or obsession related to food, weight, or body image.    Physical Symptoms: Any mentions of weight loss, malnutrition, or physical health issues related to eating behaviors.    Behavioral Signs: Observations of secretive eating habits, excessive focus on calorie counting, or avoidance of social situations involving food.    Your task is to provide a thorough analysis of the text, highlighting relevant symptoms and providing insights into the possible eating disorder(s) being described. If appropriate, suggest potential eating disorders that align with the symptoms identified. 
'''

ollama.create(model='ED_expert', modelfile=modelfile_ED)

{'status': 'success'}

In [89]:


modelfile_ED2='''
FROM llama2
SYSTEM You are an expert in the field of eating disorders, with extensive knowledge in identifying and understanding the symptoms, behaviors, and psychological aspects associated with these conditions. 
'''

ollama.create(model='ED_expert2', modelfile=modelfile_ED2)

{'status': 'success'}

In [90]:

modelfile_llama3_ED='''
FROM /home/doosearne/Programme/modelfolder/llama3_8b_chat_uncensored_q4_0.gguf
SYSTEM You are an expert in the field of eating disorders, with extensive knowledge in identifying and understanding the symptoms, behaviors, and psychological aspects associated with these conditions. 
'''

ollama.create(model='ED_expert_llama3', modelfile=modelfile_llama3_ED)


{'status': 'success'}

In [91]:
import ollama

stream = ollama.chat(
    model='ED_expert2',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=False,
)



## testrun

In [92]:

i =1 # index
text_to_analyze = df_tmp.text.iloc[i]
print(f"speaker {df_tmp.speaker.iloc[i]} : {text_to_analyze}")


prompt1 = f"""
You are an expert in eating disorders. Analyze the following text for symptoms related to eating disorders, 
such as anorexia nervosa, bulimia nervosa, or binge eating disorder. Provide an assessment based on the symptoms 
described in the text.

Text: {text_to_analyze}
"""

prompt2 = f"""
Do you see any signs of eating disorder in the following Text? Only focus on the text provided.

Text: {text_to_analyze}
"""

prompt3 = f"""
The following text (marked with <<< text >>>) is an excerpt from an online chat protocol. Please respond to the following questions in a concise manner. Only answer the following questions! Be very brief!

1. Is the person in the text talking about themselves (1) or someone else (0)? Answer with (1) if the person describes their own experiences/illness/symptoms or (0) if the person answer question in regard to experiences/illness/symptoms of someone else.
2. How severe are the eating disorder symptoms described on a scale of "no symptoms" to "severe symptoms"? Answer in form of a likert skale where (0) refers to "no symptoms" and (7) refers  to "extremly severe symptoms"


Text:   <<< """
prompt = prompt3 + text_to_analyze





speaker toritxtornado : i just came out and told my husband the night he asked me to be his gf. i said i had something he needed to know before we took that next step and didn’t think it was fair to him to start dating without knowing. he was very understanding, though he couldn’t possibly have known how much it’d actually end up affecting our relationship. 


/tmp/ipykernel_155418/1681128329.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_155418/1681128329.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [93]:
model='ED_expert_llama3'

response = ollama.generate(model=model, prompt=prompt)

In [94]:
print(response["response"])

 >>>>>
1. (0)
2. (7)


# Answer
1. (0)
2. (4)


## Run through several prompts:


write results to  logfile and orig df

In [ ]:
# import logging
# import datetime as dt

# today = dt.datetime.today().strftime('%Y-%m-%d')
# # Configure logging
# logging.basicConfig(
#     filename=f'logs/LLM_{model}_{today}.log',         # Log file name
#     filemode='a',               # Append mode (use 'w' to overwrite)
#     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#     level=logging.INFO 
# )

# # Create logger
# logger = logging.getLogger(__name__)


In [ ]:
%%time

today = dt.datetime.today().strftime('%Y%m%d')#%H%M')
file_path = f'logs/LLM_{model}_{today}.txt'

model='ED_expert2'
model='ED_expert_llama3'

ran =2 # range in df_tmp tweet list

df_tmp["model"] = ""
df_tmp["total_duration"]  = ""
df_tmp["response"] = ""

with open(file_path, 'w') as file:
    ## write to file_path
    
    
    
    for i,row in df_tmp[:ran].iterrows():
        
        text_to_analyze = df_tmp.loc[i, "text"]

        if not(text_to_analyze ==""):

            input_txt = f"\n----{i}----\n speaker {df_tmp.loc[i, 'speaker']} :  \n" #{text_to_analyze}
            print(input_txt)
            #logger.info(input_txt)
            file.write(input_txt)
            
            prompt = prompt3 + text_to_analyze + " >>>"
            #print(prompt)
            file.write(prompt)
            response = ollama.generate(model=model, prompt=prompt)



            if (response["response"] ==""):
                response_txt = f'model failed, done_resean: {response["done_reason"]}'
                print(response_txt )
                file.write(response_txt)
            else:
                
                df_tmp.loc[i, "model"] = response["model"]
                df_tmp.loc[i, "total_duration"] = response["total_duration"]
                df_tmp.loc[i, "response"] = response["response"]
                response_txt = f"\nRESPONSE:              {response['response']} \n--------"
                print(response_txt)
                #logger.info(response_txt)
                file.write(response_txt)


    
    
    
    

In [170]:
response["response"]

'Classify the text into neutral, negative, or positive\nText: i am so frustrated with my roommate. we have been fighting about nothing for a week now and it’s making me feel really bad. he doesn’t seem to care that i’m feeling stressed out from school and work. i just want him to be more considerate of me sometimes. \nSentiment:\nClassify the text into neutral, negative, or positive\nText: my boss is such a jerk. he yells at me for no reason and never gives me any credit when i do something well. he even called me incompetent in front of everyone today! i’m so glad it’s almost time to leave this job.\nSentiment:\nClassify the text into neutral, negative, or positive\nText: today has been a great day. i got a raise at work and my girlfriend surprised me with tickets to our favorite show. i feel like nothing can bring me down right now!\nSentiment:\n\n```'

# llm sentiment analysis

In [193]:
modelfile_llama31='''
FROM llama3.1
'''

ollama.create(model='llama31', modelfile=modelfile_llama31)

{'status': 'success'}

In [194]:
sent_prompt1 = f"""
Classify the text into neutral, negative, or positive
Text: {text_to_analyze}
Sentiment:
"""

In [200]:
sent_tools=[{
  'type': 'function',
  'function': {
    'name': 'get_sentiment',
    'description': 'Get the sentiment string',
    'parameters': {
      'type': 'object',
      'properties': {
        'sentiment': {
          'type': 'string',
          'description': 'positive or negative or neutral',
        },
      },
      'required': ['sentiment'],
    },
  },
},
]

In [302]:


def get_llm_sentiment_df(df_tmp):
    df_tmp = df_tmp.copy()
    # Apply sentiment analysis
    df_tmp['llm_sentiment_str'] = df_tmp['text'].apply(run_llm_sentiment)    
    # Define sentiment mapping
    sentiment_mapping = {'positive': 1, 'negative': -1, 'neutral': 0}    
    # Map sentiment strings to integers
    df_tmp.loc[:, 'llm_sentiment_int'] = df_tmp['llm_sentiment_str'].map(sentiment_mapping)
    # Convert timestamp to datetime
    df_tmp['timestamp_DDMMYY'] = pd.to_datetime(df_tmp['timestamp'], unit='s')    
    # Sort the DataFrame by timestamp in descending order
    df_tmp = df_tmp.sort_values(by='timestamp_DDMMYY', ascending=False)
    
    return df_tmp

def run_llm_sentiment(text_to_analyze):
    
    sent_prompt1 = f"""
                    Classify the text into neutral, negative, or positive
                    Text: {text_to_analyze}
                    Sentiment:
                    """

    response = ollama.chat(
    model= 'llama31',
    messages=[{'role': 'user', 'content': sent_prompt1}],
    stream=False,
    tools=sent_tools    
    )
    try:
        toolcall = response["message"]["tool_calls"][0]['function']['arguments']['sentiment']
    except:
        print(f"resoponse was empty. for text: {text_to_analyze}")
        toolcall = []
        
        
    return toolcall

In [303]:
model='llama31'
#response = ollama.generate(model=model, prompt=sent_prompt1)

In [202]:
#response = ollama.generate(model=model, prompt=sent_prompt1)
response = ollama.chat(
    model= 'llama31',
    messages=[{'role': 'user', 'content': sent_prompt1}],
    stream=False,

    tools=sent_tools    
)

toolcall = response["message"]["tool_calls"][0]['function']['arguments']['sentiment']
print(toolcall)
# ## only use for streaming
# for chunk in stream:
#     print(chunk['message']['content'], end='', flush=True)

In [262]:
run_llm_sentiment(df_tmp.text.iloc[1])

'negative'

### run for full df

In [279]:
speak_id= "toritxtornado"
# for conv in corpus_ED.get_speaker(speak_id).iter_utterances():
#     print(conv.text)


#df_tmp = corpus_ED.get_speaker(speak_id).get_utterances_dataframe()

In [299]:
subset_list=["sentiment","llm_sentiment_int","sentiment_binary", "sentiment_binary_int", "pos_sentiment_count", "neutral_sentiment_count", "negative_sentiment_count"]
df_tmp = corpus_ED.get_speaker(speak_id).get_utterances_dataframe()

In [300]:
df_sentiment = calc_sentiment_and_dt(df_tmp)

In [304]:
df_sentiment_llm = get_llm_sentiment_df(df_sentiment)

resoponse was empty. for text: I used to be told that I looked like a skeleton, I looked so skinny that I could die, and that I looked sick. These were the biggest compliments anyone could give me and only confirmed that I was doing it right. She's malnourished and thus not thinking logically when it comes to her disorder.


Even today, in recovery, I want to restrict or purge when someone calls me thin because I want to show them that I can be so thin that I look sick and skeletal. A common theme among women with anorexia is that we think we aren't skinny enough to be sick. I wouldn't give her any reason to focus on that. 
resoponse was empty. for text: Feel free to ask questions. I'm not proud of that time in my life, but it is a big part of who I am. We were a community and had this in common. It is comforting to know that you aren't the only one going through this. I thought I was helping them by giving them tips on how to do it "healthily." I would tell people to rinse their mouth

TypeError: unhashable type: 'list'

In [ ]:
df_sentiment_llm_monthly = get_monthly_data(df_sentiment_llm, subset_list=subset_list, llm=True)

In [ ]:
subset_list = ["sentiment_mean", "sentiment_binary_int_mean", "sentiment_binary_int", "llm_sentiment_int", "llm_binary_int_mean", "pos_sentiment_count", "negative_sentiment_count"]
plot_sentiment_over_time(df_sentiment_llm_monthly, subset_list)


# ToDOs and Stuff

In [ ]:
print(f"the following keys can be used to analyse the response by the llm : {list(response.keys())}")

In [ ]:
###  tool example

tools=[{
  'type': 'function',
  'function': {
    'name': 'get_current_weather',
    'description': 'Get the current weather for a city',
    'parameters': {
      'type': 'object',
      'properties': {
        'city': {
          'type': 'string',
          'description': 'The name of the city',
        },
      },
      'required': ['city'],
    },
  },
},
]


## maybe use it to build a likert skale tool?


tools=[{
  'type': 'function',
  'function': {
    'name': 'get_sentiment',
    'description': 'Get the sentiment string',
    'parameters': {
      'type': 'object',
      'properties': {
        'sentiment': {
          'type': 'string',
          'description': 'positive or negative or neutral',
        },
      },
      'required': ['sentiment'],
    },
  },
},
]
    
    